# Task 1: Third-order letter approximation model

# Import necessary libraries.

In [ ]:
import os
from collections import defaultdict

# Step 1: Format text files
# Begin by assigning the path to the folder containing the texts to a variable

In [ ]:
# Stores path to folder containing the texts
englishTexts = r'C:\Users\Luke\OneDrive - Atlantic TU\Yr2Semester1\Desktop\EmergingTechG00410404\docs'
